# Transfer Learning

Proprietary material - Under Creative Commons 4.0 licence CC-BY-NC-ND https://creativecommons.org/licenses/by-nc-nd/4.0/



# Introduction

This week we will cover the basics of how to apply Transfer Learning to a Neural Net in Keras.

Most, if not all the the big Deep Learning libraries come with some tools to apply Transfer Learning from another model. Some even include models trained and ready for deployment that can be used directly for Transfer Learning, and luckly for us, Keras includes that option.

## Layer Freezing

Before using Transfer Learning, its necessary to go over an intruduction on how to freeze a model layers and what does that mean.

In [ ]:
# Importing relevant libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16

First, lets load a Neural Net from the Keras library. 

The architecture is an Xception Net and trained in the ImageNet dataset.



In [ ]:
vgg_net = VGG16(weights='imagenet', input_shape=(224, 224, 3), include_top=True) 
vgg_net.summary()

553467904/553467096 [==============================] - 5s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

A frozen layer in a model means that the layer weights wont be updated during training, so to apply transfer learning you have to control which layers need to be frozen or unfrozen before or in the middle of training.

To count the number of unfrozen or frozen weights, we can access the lists '*trainable\_weights*' and  '*non\_trainable\_weights*' respectively.

**Note:** Notice that the weights are 32, which is the double of the number of layers of the model (VGG16). This is because for each FC and Conv layer there is two weights (the kernel and bias).

In [ ]:
print("Number of weights:", len(vgg_net.weights))
print("Number of trainable weights:", len(vgg_net.trainable_weights))
print("Number of frozen weights:", len(vgg_net.non_trainable_weights))

Number of weights: 32
Number of trainable weights: 32
Number of frozen weights: 0


We can freeze a layer by setting the layer '*trainable*' to False.
As a test, lets freeze the layers 'block1\_conv2' and 'block4\_conv3' in the positions 3 and 13 of the layers list.

In [ ]:
vgg_net.layers[2].trainable = False
vgg_net.layers[13].trainable = False

Now lets check that those layers are frozzen.

In [ ]:
print("Number of weights:", len(vgg_net.weights))
print("Number of trainable weights:", len(vgg_net.trainable_weights))
print("Number of frozen weights:", len(vgg_net.non_trainable_weights))

Number of weights: 32
Number of trainable weights: 28
Number of frozen weights: 4


Seting the trainable to False works on all the sublayers, so if we want to freeze all layers we can do this:

In [ ]:
vgg_net.trainable = False

print("Number of weights:", len(vgg_net.weights))
print("Number of trainable weights:", len(vgg_net.trainable_weights))
print("Number of frozen weights:", len(vgg_net.non_trainable_weights))

Number of weights: 32
Number of trainable weights: 0
Number of frozen weights: 32


The summary also indicates how many parametres can be trained in a model (at the bootom). 

In [ ]:
vgg_net.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

# Transfer Learning

Now lets finally prepare our model to train with Transfer Learning!

First, lets grab one of the CNNs that come with Keras with no FC layers at the end.

This can be done by passing the 'include\_top' aprameter as False.

In [ ]:
vgg_net = VGG16(weights='imagenet', input_shape=(224, 224, 3), include_top=False) 
vgg_net.trainable = False
vgg_net.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

Now lets add a couple of FC layers on top. For now, lets asume our otput is a binary classifier for simplicity.

In [ ]:
from keras.models import Sequential 
from keras.layers import Dense, Flatten
from keras import optimizers

model = Sequential()

model.add(vgg_net)
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_5 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 4096)              102764544 
_________________________________________________________________
dense_13 (Dense)             (None, 1024)              4195328   
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 1025      
Total params: 121,675,585
Trainable params: 106,960,897
Non-trainable params: 14,714,688
_________________________________________________________________


Now we have our model ready for the training loop!

But lets leave that for the problem ;)

Before geting to that, try to keep in mind the following considerations:



1.   To accelerate the training, you can extract the features from the data using the frozen layers and train the rest of the model using those features.

2.   The previos point can't be used if you are using data augmentation douring the training, but it can be done beforehand.

3.  At the end of the training, its recomended to apply a bit of Fine Tunning to the model, by unfreezing the model and reducing the learning rate.

4.  You can't add new layers before the frozen layers. If you need to, Fine Tuning is the only option.

5. When using a CNN for transfer Learning, remember to modify your data so it has the expected size for the model.






## Exercise 

Lets build a cat vs dog image classifier with a CNN. For that, you are free to so as you want, but we recomend at leats some of the following steps:

1. Download the data from this link https://www.kaggle.com/c/dogs-vs-cats/data. If you want to use some other dataset, you are free to do so.

2. Explore the data, visualize some examples and check the size of the images.

3.  Choose an architecture from https://keras.io/api/applications/ to your liking. 

4. Reshape the images to the same size. For this you can use OpenCV like this:



```
import cv2
import os

path = 'path tho directory'
image_paths = os.listdir(path)

for file in image_paths:
    image = cv2.imread(os.path.join(path, file))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (224, 224))

    ...
```

Then you can save the images to a list or data structure inside the loop. Some files include their class in the name, so this loop offers a great oportunity to prepare the labels too.



4. If the shape of the data is diferent to the ImageNet data, you need to reshape the input. For this you can add the parameter 'input\_tensor' when loading the trained model. For example, for an input of 128x128:



```
vgg_net = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(128, 128, 3)))
```

5. Freeze the model and add some new FC and Dropout layers with apropiate sizes. 

5. Separate the data into train and validation by 80% train and 20% validation. 

6. Train the model over the train data. Use the Adam optimizer and choose a loss apropiate to your problem. As an example, in case its a binary classifier use  Binary Cross Entropy.

7. Optional: Unfreeze the model at the end and train once again the model using Fine Tuning.

8. Analize the training time and the performance of the model in the test data. Optional: Share some of you results with the team and discuss about the architecture, loss and performace of your model. 


We understand that some of this steps might be a bit complicated or unclear for some, so dont be afraid to ask questions to the team in the public channels or check the references for some guidance.  



## References


The oficial Keras Tutorial

https://keras.io/guides/transfer_learning/

List and examples of some architectures in Keras 

https://keras.io/api/applications/

A great tutorial that goes deeper into Transfer Learning, but we recomend checking it out after being done with the exercise. And I trust you will.

https://towardsdatascience.com/a-comprehensive-hands-on-guide-to-transfer-learning-with-real-world-applications-in-deep-learning-212bf3b2f27a